In [ ]:
!pip install transformers &>/dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import sys
import os
py_file_location = "/content/drive/MyDrive/EdgeML_Team/bert-entity-extraction/src_atharva"
sys.path.append(os.path.abspath(py_file_location))

import joblib
import torch
import config
import dataset
import engine
from model import EntityModel

In [ ]:
  meta_data = joblib.load("/content/drive/MyDrive/EdgeML_Team/bert-entity-extraction/Models/meta_atharva.bin")
  enc_pos = meta_data["enc_pos"]
  enc_tag = meta_data["enc_tag"]
  num_pos = len(list(enc_pos.classes_))
  num_tag = len(list(enc_tag.classes_))

In [ ]:
  device = torch.device('cuda')
  model = EntityModel(num_tag=num_tag, num_pos=num_pos)
  model.load_state_dict(torch.load(config.MODEL_PATH))
  model.to(device)
  print()

Some weights of the model checkpoint at /content/drive/MyDrive/BERT/input/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_tags(test_dataset):
  with torch.no_grad():
        data = test_dataset[0]
        # print(data)
        for k, v in data.items():
            data[k] = v.to(device).unsqueeze(0)
        tag, pos, _ = model(**data)
        return tag, pos,_

In [ ]:
def get_tokens(tags,tokenized_sentence,tags_name):
  map_ = {}
  for i in enc_tag.classes_:
    map_[i] = []

  for i in range(256):
    if tags[i] != 16 and tokenized_sentence[i] != 101:
      map_[tags_name[i]].append(tokenized_sentence[i])
  return map_;

In [ ]:
final_info = {"Account_no" : [], "Admit_Date" : [], "MR_Number" : [], "Patient_Name" : [],  "Social_Security_no" : [], "Age" : [], "DOB" : [], "Patient_Phone_no" : [], "Admitting_Disease" : [], "Admitting_Physician" : [], "Primary_Insurance_Policy" : []}
def get_mapping(sentence):
    extracted_info = {"Account_no" : [], "Admit_Date" : [], "MR_Number" : [], "Patient_Name" : [],  "Social_Security_no" : [], "Age" : [], "DOB" : [], "Patient_Phone_no" : [], "Admitting_Disease" : [], "Admitting_Physician" : [], "Primary_Insurance_Policy" : []}
    tokenized_sentence = config.TOKENIZER.encode(sentence)
    sentence = sentence.split()
    test_dataset = dataset.EntityDataset(
            texts=[sentence], 
            pos=[[0] * len(sentence)], 
            tags=[[0] * len(sentence)]
    )
    tag,pos,_ = get_tags(test_dataset)

    tags = tag.argmax(2).cpu().numpy().reshape(-1)

    tags_name = enc_tag.inverse_transform(
                    tag.argmax(2).cpu().numpy().reshape(-1)
                )[:len(tokenized_sentence)]
    #print(len(tokenized_sentence))
    map_ = get_tokens(tags,tokenized_sentence,tags_name)

    for i in map_:
        #print(i,"-->",config.TOKENIZER.decode(map_[i]))
        if i == "B-ACCOUNT #":
          extracted_info['Account_no'].append(config.TOKENIZER.decode(map_[i]))
          final_info['Account_no'].append(extracted_info['Account_no'][0])

        if i == "B-ADMIT DATE":
          extracted_info['Admit_Date'].append(config.TOKENIZER.decode(map_[i]))
          final_info['Admit_Date'].append(extracted_info['Admit_Date'][0])

        if i == "B-ADMITTING DIAGNOSIS":
          extracted_info['Admitting_Disease'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Admitting_Disease'][0] = extracted_info['Admitting_Disease'][0].split()[0]
        if i == "I-ADMITTING DIAGNOSIS":
          extracted_info['Admitting_Disease'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Admitting_Disease'][0] =' '.join(extracted_info['Admitting_Disease'])
          extracted_info['Admitting_Disease'].pop()
          final_info['Admitting_Disease'].append(extracted_info['Admitting_Disease'][0])

        if i == "B-ADMITTING PHYSICIAN":
          extracted_info['Admitting_Physician'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Admitting_Physician'][0] = extracted_info['Admitting_Physician'][0].split()[0]
        if i == "I-ADMITTING PHYSICIAN":
          extracted_info['Admitting_Physician'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Admitting_Physician'][0] =' '.join(extracted_info['Admitting_Physician'])
          extracted_info['Admitting_Physician'].pop()
          final_info['Admitting_Physician'].append(extracted_info['Admitting_Physician'][0])

        if i == "B-AGE":
          extracted_info['Age'].append(config.TOKENIZER.decode(map_[i]))
          final_info['Age'].append(extracted_info['Age'][0])

        if i == "B-BIRTH DATE":
          extracted_info['DOB'].append(config.TOKENIZER.decode(map_[i]))
          final_info['DOB'].append(extracted_info['DOB'][0])

        if i == "B-MR NUMBER":
          extracted_info['MR_Number'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['MR_Number'][0] = extracted_info['MR_Number'][0].replace(" ", "")
          extracted_info['MR_Number'][0] = extracted_info['MR_Number'][0][:8]
          final_info['MR_Number'].append(extracted_info['MR_Number'][0])

        if i == "B-PATIENT NAME":
          extracted_info['Patient_Name'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Patient_Name'][0] = extracted_info['Patient_Name'][0].split()[0]
        if i == "I-PATIENT NAME":
          extracted_info['Patient_Name'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Patient_Name'][0] =' '.join(extracted_info['Patient_Name'])
          extracted_info['Patient_Name'].pop()
          final_info['Patient_Name'].append(extracted_info['Patient_Name'][0])

        if i == "B-PATIENT PHONE #":
          extracted_info['Patient_Phone_no'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Patient_Phone_no'][0] = extracted_info['Patient_Phone_no'][0].replace("(", "")
          extracted_info['Patient_Phone_no'][0] = extracted_info['Patient_Phone_no'][0].replace(")", "")
          extracted_info['Patient_Phone_no'][0] = extracted_info['Patient_Phone_no'][0].replace(".", "")
          extracted_info['Patient_Phone_no'][0] = extracted_info['Patient_Phone_no'][0].replace(" ", "")
          extracted_info['Patient_Phone_no'][0] = extracted_info['Patient_Phone_no'][0].replace("-", "")
          extracted_info['Patient_Phone_no'][0] = extracted_info['Patient_Phone_no'][0][:10]
          final_info['Patient_Phone_no'].append(extracted_info['Patient_Phone_no'][0])

        if i == "B-PRIMARY INSURANCE PLAN":
          extracted_info['Primary_Insurance_Policy'].append(config.TOKENIZER.decode(map_[i]))
          if extracted_info['Primary_Insurance_Policy'] == "":
            extracted_info['Primary_Insurance_Policy'] = "-"
            extracted_info['Primary_Insurance_Policy'][0] = extracted_info['Primary_Insurance_Policy'][0].split()[0]
        if i == "I-PRIMARY INSURANCE PLAN":
          extracted_info['Primary_Insurance_Policy'].append(config.TOKENIZER.decode(map_[i]))
          if extracted_info['Primary_Insurance_Policy'][1] == "":
            extracted_info['Primary_Insurance_Policy'][1] = "-"
          extracted_info['Primary_Insurance_Policy'][0] =' '.join(extracted_info['Primary_Insurance_Policy'])
          extracted_info['Primary_Insurance_Policy'].pop()

          final_info['Primary_Insurance_Policy'].append(extracted_info['Primary_Insurance_Policy'][0])

        if i == "B-SOCIAL SECURITY #":
          extracted_info['Social_Security_no'].append(config.TOKENIZER.decode(map_[i]))
        if i == "I-SOCIAL SECURITY #":
          extracted_info['Social_Security_no'].append(config.TOKENIZER.decode(map_[i]))
          extracted_info['Social_Security_no'][0] =' '.join(extracted_info['Social_Security_no'])
          extracted_info['Social_Security_no'].pop()

          extracted_info['Social_Security_no'][0] = extracted_info['Social_Security_no'][0].replace(" ", "")
          extracted_info['Social_Security_no'][0] = extracted_info['Social_Security_no'][0].replace("-", "")
          extracted_info['Social_Security_no'][0] = extracted_info['Social_Security_no'][0].replace("(", "")
          extracted_info['Social_Security_no'][0] = extracted_info['Social_Security_no'][0].replace(")", "")
          extracted_info['Social_Security_no'][0] = extracted_info['Social_Security_no'][0][:9]
          #extracted_info['Social_Security_no'][0] = int(extracted_info['Social_Security_no'][0])
          final_info['Social_Security_no'].append(extracted_info['Social_Security_no'][0])
        
    #return final_info

In [ ]:
import glob

TEXT_FILE_PATH = r'/content/drive/MyDrive/EdgeML_Team/bert-entity-extraction/Dataset/Text /*.txt'

for i in glob.glob(TEXT_FILE_PATH):

  f = open(i, "r")

  sentence = f.read();

  #print(sentence)

  get_mapping(sentence)

383
370
376
398
391
383
399
389
380
380


In [ ]:
import pandas as pd
extracted_df = pd.DataFrame(final_info)

In [ ]:
extracted_df

,Account_no,Admit_Date,MR_Number,Patient_Name,Social_Security_no,Age,DOB,Patient_Phone_no,Admitting_Disease,Admitting_Physician,Primary_Insurance_Policy
0,61138,02 / 17 / 202,74010653,mills pi lake aaron,555183400,70,07 / 21 / 19,83,asthma,jonathan parker,northern care
1,72645,06 / 21 / 201,99065239,roberts pi,864375912,75,03 / 21 / 19,1901473824,breast disease,courtney patterson contre,pacific eastern care care
2,02487,02 / 24 / 202,54391500,robertson jeffrey,066586681,20,01 / 07 / 201 /,4469529017,prostate disease,christopher johnson,northern care
3,68907,09 / 14 / 201,39954861,kidd brandi,439968298,92,09 / 07 / 19,300,retinal ischemia,monica miller,-
4,99784,10 / 28 / 201,43968308,moore jason,049580467,85,10 / 08 / 19 :,60814677,herpes simplex,carol,northern care
5,97824,10 / 15 / 201,72580097,jones danielle,249405537,55,03 / 13 / 191,4598473930,cancer,combs hampton,western care
6,89410,12 / 07 / 201,66103393,davidson jennifer,106871289,58,09 / 11 / 191,06481,alopecia area,jimmy small,-
7,03880,04 / 29 / 202,28611094,reeves brian,275389492,57,06 / 19 / 191,066,gingivitis surgery,bradley mcdowell,southern care
8,84704,03 / 14 / 201,77879406,brown stephanie,350775915,11,/ 25 / 20u,9991943,leukemia surgery,douglas estrada,eastern care
9,41203,10 / 02 / 201,78422297,smith brittany,857525075,96,02 / 07 / 19,##40,atheroscle ##rosis,phillip garcia,northern care


In [ ]:
extracted_df.to_csv('/content/drive/MyDrive/EdgeML_Team/bert-entity-extraction/Results/extracted_results.csv', header = True, index = False)

In [ ]:
!python /content/drive/MyDrive/EdgeML_Team/bert-entity-extraction/src/prediction_with_mapping.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2021-05-12 05:32:49.693980: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
